# Chapter 09-01: 분산 학습 기초 (Distributed Training Basics)

## 학습 목표
- 단일 GPU의 **메모리 한계**를 수치로 이해하고, 분산 학습이 필요한 이유를 설명할 수 있다
- **Data Parallelism(DP)**와 **Distributed Data Parallel(DDP)**의 구조와 성능 차이를 수식으로 비교한다
- **Ring All-Reduce** 알고리즘의 통신 비용 수식 $T = 2(N-1)\frac{K}{N}$을 스스로 도출하고, 순수 파이썬으로 구현한다
- TensorFlow `MirroredStrategy`를 사용하여 단일 노드 분산 학습 파이프라인을 완성한다

## 목차
1. [수학적 기초: 통신 비용과 확장성](#1.-수학적-기초)
2. [왜 분산 학습이 필요한가? (메모리 계산)](#2.-메모리-계산)
3. [DP vs DDP 구조 비교](#3.-DP-vs-DDP)
4. [Ring All-Reduce 알고리즘 심화](#4.-Ring-All-Reduce-심화)
5. [TF MirroredStrategy 실전 실습](#5.-MirroredStrategy-실습)
6. [통신 비용 시각화 비교](#6.-통신-비용-시각화)
7. [정리 및 연습 문제](#7.-정리)

## 1. 수학적 기초

### 통신 비용 (All-Reduce)

Ring All-Reduce를 이해하기 위한 핵심 수식:

$$\boxed{T_{comm} = 2(N-1)\frac{K}{N} \cdot t_{byte}}$$

- $N$: GPU(노드) 수
- $K$: 전체 그래디언트 텐서의 바이트 크기
- $t_{byte}$: 바이트당 전송 시간 (네트워크 대역폭의 역수)

**핵심**: $N \to \infty$일 때 $T_{comm} \to 2K \cdot t_{byte}$, 즉 GPU가 늘어도 통신 비용이 **상수**로 수렴!

### 학습 메모리 공식 (Mixed Precision + Adam)

| 항목 | 데이터타입 | 바이트 |
|------|-----------|--------|
| 모델 파라미터 | FP16 | $2P$ |
| 그래디언트 | FP16 | $2P$ |
| 마스터 가중치 | FP32 | $4P$ |
| Adam 1차 모멘트 $m$ | FP32 | $4P$ |
| Adam 2차 모멘트 $v$ | FP32 | $4P$ |
| **합계** | | $\mathbf{16P}$ bytes |

> 예: Llama-2 7B → $16 \times 7 \times 10^9 \approx 112$ GB

---

### 🐣 초등학생을 위한 친절 설명!

#### 🏭 분산 학습이 왜 필요한가요?

AI 모델을 훈련시키는 것은 마치 **엄청나게 큰 퍼즐을 맞추는 것**과 같아요.

```
🧩 퍼즐 조각 수: 70,000,000,000개 (Llama-2 70B의 파라미터 수!)
🗂️ 방 크기 (GPU): 80GB (아무리 큰 방도 결국 꽉 찬다!)
```

> 💡 **비유**: 방 하나에 퍼즐이 안 들어가면? 여러 방으로 나눠서 각자 담당 구역을 맞추면 됩니다!  
> 하지만 마지막에 합쳐야 하므로, 요리사들끼리 **레시피(그래디언트)**를 공유해야 해요.

#### 🔄 All-Reduce가 뭔가요?

> 4팀이 각자 부분 점수를 계산한 후, 모든 팀이 총점을 알아야 할 때:
> - **나쁜 방법**: 반장(마스터 GPU)에게 다 전달 → 반장이 더함 → 다시 배포  
>   (반장이 너무 바빠짐!)
> - **좋은 방법**: 1팀→2팀→3팀→4팀→1팀 순서로 돌아가며 합산  
>   (모든 팀이 일을 나눔! = **Ring All-Reduce**)

### 📝 연습 문제

#### 문제 1: 필요 GPU 수 계산

GPT-3 175B 모델을 H100 80GB GPU로 학습한다. Mixed Precision + Adam 기준,  
필요한 최소 GPU 수는? (각 GPU를 80% 효율로 사용한다고 가정)

<details>
<summary>💡 풀이 확인</summary>

총 학습 메모리 = $16 \times 175 \times 10^9$ bytes $= 2800$ GB  
GPU당 활용 메모리 = $80 \times 0.8 = 64$ GB  
필요 GPU = $\lceil 2800 / 64 \rceil = \mathbf{44}$대

(실제로는 Activation, Temp 메모리 등으로 더 필요 → 실제 학습엔 1,024~4,096대 사용)
</details>

#### 문제 2: Ring All-Reduce 통신량 계산

128대 GPU가 FP32 그래디언트($4K$ bytes, $K = 175 \times 10^9$)를 All-Reduce 할 때,  
GPU당 총 통신량을 계산하라. (단순 마스터 중심 방식과 비교할 것)

<details>
<summary>💡 풀이 확인</summary>

그래디언트 크기 $K_{bytes} = 175 \times 10^9 \times 4 = 700$ GB  

**Ring All-Reduce**: $2(128-1) \times \frac{700}{128} \approx 2 \times 700 = 1400$ GB (수렴값)

**마스터 중심 Gather**: 마스터가 $127 \times 700 = 88900$ GB 수신 → **63.5배** 더 많음!
</details>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 2. 메모리 계산

In [ ]:
# ---------------------------------------------------
# 모델 크기별 학습 메모리 및 필요 GPU 수 계산
# ---------------------------------------------------

def compute_training_memory(params_B, optimizer='adam', mixed_precision=True):
    """Mixed Precision + 옵티마이저 기준 학습 메모리 계산 (GB)"""
    P = params_B * 1e9
    if mixed_precision:
        model_gb  = P * 2 / 1e9  # FP16 모델
        grad_gb   = P * 2 / 1e9  # FP16 그래디언트
        master_gb = P * 4 / 1e9  # FP32 마스터 가중치
    else:
        model_gb  = P * 4 / 1e9
        grad_gb   = P * 4 / 1e9
        master_gb = 0.0

    optim_gb = {'sgd': 0, 'momentum': P*4/1e9, 'adam': P*8/1e9}[optimizer]
    total_gb = model_gb + grad_gb + master_gb + optim_gb
    return {'model': model_gb, 'grad': grad_gb, 'master': master_gb,
            'optim': optim_gb, 'total': total_gb}

models = [
    ('BERT-L\n0.34B', 0.34), ('GPT-2\n1.5B', 1.5),
    ('Llama-2\n7B', 7),   ('Llama-2\n70B', 70),
    ('GPT-3\n175B', 175)
]
gpu_vram = 80  # H100 80GB

print(f"{'모델':<15} {'총 메모리(GB)':>14} {'H100 80GB 필요 수':>18}")
print("-" * 50)
for label, p in models:
    m = compute_training_memory(p)
    n_gpu = max(1, int(np.ceil(m['total'] / (gpu_vram * 0.8))))
    print(f"{label.replace(chr(10), ' '):<15} {m['total']:>14.1f} GB {n_gpu:>16} 대")

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

labels = [m[0].replace('\n', ' ') for m in models]
totals = [compute_training_memory(m[1])['total'] for m in models]
results = [compute_training_memory(m[1]) for m in models]
colors = ['#42A5F5', '#66BB6A', '#FFA726', '#EF5350', '#AB47BC']

# 스택 바
ax1 = axes[0]
x = np.arange(len(labels))
w = 0.55
b0 = [r['model'] for r in results]
b1 = [r['grad'] for r in results]
b2 = [r['master'] for r in results]
b3 = [r['optim'] for r in results]

ax1.bar(x, b0, w, label='모델 FP16', color='#2196F3')
ax1.bar(x, b1, w, bottom=b0, label='그래디언트 FP16', color='#4CAF50')
ax1.bar(x, b2, w, bottom=[a+b for a,b in zip(b0,b1)], label='마스터 FP32', color='#FF9800')
ax1.bar(x, b3, w, bottom=[a+b+c for a,b,c in zip(b0,b1,b2)], label='Adam FP32', color='#F44336')
ax1.axhline(80, color='darkred', linestyle='--', lw=2, label='H100 80GB')
ax1.set_yscale('log')
ax1.set_xticks(x); ax1.set_xticklabels(labels, fontsize=9)
ax1.set_ylabel('메모리 (GB, 로그 스케일)')
ax1.set_title('Mixed Precision + Adam 학습 메모리', fontweight='bold')
ax1.legend(fontsize=8, loc='upper left'); ax1.grid(True, alpha=0.3, axis='y')

# 구성 비율 (7B 기준)
ax2 = axes[1]
r7 = compute_training_memory(7)
sizes = [r7['model'], r7['grad'], r7['master'], r7['optim']]
lbls = [f'모델 FP16\n{r7["model"]:.0f}GB',
        f'그래디언트 FP16\n{r7["grad"]:.0f}GB',
        f'마스터 FP32\n{r7["master"]:.0f}GB',
        f'Adam FP32\n{r7["optim"]:.0f}GB']
ax2.pie(sizes, labels=lbls, colors=['#2196F3','#4CAF50','#FF9800','#F44336'],
        autopct='%1.0f%%', startangle=90, textprops={'fontsize': 9})
ax2.set_title(f'Llama-2 7B 메모리 구성 (총 {r7["total"]:.0f}GB)', fontweight='bold')

plt.tight_layout()
plt.show()
print(f"\n→ Adam 옵티마이저 상태(FP32)가 전체의 {r7['optim']/r7['total']*100:.0f}%를 차지!")
print("→ 이것이 ZeRO Stage 1(옵티마이저 상태 분산)부터 시작하는 이유다.")

## 3. DP vs DDP

### Data Parallelism (DP) — 단일 프로세스 방식

```
                  ┌──────────────────────────────────────┐
                  │         마스터 GPU (GPU 0)           │
               ①  │  전체 배치 수신 → 모델 복사본 보유    │
                  └──────┬──────────────────────┬────────┘
           ②Scatter      │                      │  ②Scatter
                  ┌──────┴───────┐        ┌─────┴────────┐
                  │   GPU 1     │        │    GPU 2     │
                  │ Forward+Bwd │        │  Forward+Bwd │
                  └──────┬───────┘        └─────┬────────┘
           ③Gather       │                      │  ③Gather
                  └──────┴──────────────────────┘
                  ④ 마스터가 그래디언트 합산 → 파라미터 업데이트
                  ⑤ Broadcast: 업데이트된 파라미터를 GPU 1,2로 복사
```

**마스터 병목**: GPU가 N개이면 마스터는 N-1개로부터 데이터를 받아야 함 → $O(N)$ 병목

### Distributed Data Parallel (DDP) — 멀티 프로세스 방식

```
    GPU 0 ←──── Ring All-Reduce ────→ GPU 1
      ↑                                  ↑
      └────────────────────────────────  ┘
    GPU 3 ←──── Ring All-Reduce ────→ GPU 2
    
    각 GPU: 독립 프로세스 → 자신의 모델 복제본 보유
    Forward / Backward → All-Reduce(그래디언트 동기화) → 로컬 업데이트
```

**마스터 없음**: 모든 GPU가 동등하게 참여 → $O(K)$ (모델 크기에만 의존)

In [ ]:
# ---------------------------------------------------
# DP vs DDP 통신 비용 수식 비교
# ---------------------------------------------------

N_range = np.arange(2, 129)  # GPU 수 2~128
K = 1.0  # 모델 크기 정규화 (1.0 = 모델 전체 크기)

# DP: 마스터 GPU가 받는 총 통신량 = (N-1) × K (Gather) + (N-1) × K (Broadcast)
dp_master_comm = 2 * (N_range - 1) * K

# DDP(Ring All-Reduce): 각 GPU의 통신량 = 2×(N-1)/N × K → 2K에 수렴
ring_comm = 2 * (N_range - 1) / N_range * K

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(N_range, dp_master_comm, 'r-', lw=2.5, label='DP: 마스터 GPU 통신량 (O(N))', zorder=3)
ax1.plot(N_range, ring_comm, 'b-', lw=2.5, label='DDP Ring All-Reduce (≈ 2K)', zorder=3)
ax1.axhline(y=2*K, color='blue', linestyle=':', alpha=0.5)
ax1.fill_between(N_range, ring_comm, dp_master_comm, alpha=0.08, color='red',
                 label='DP가 낭비하는 통신 비용')
ax1.set_xlabel('GPU 수 (N)')
ax1.set_ylabel('GPU당 통신량 (K 단위, K=모델 전체 크기)')
ax1.set_title('통신 방식별 GPU당 통신량 비교', fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.annotate('N=64일 때\nDP: 126K vs Ring: ≈2K → 63배 차이!',
             xy=(64, 126), xytext=(70, 90),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=9, color='red')

# 이상적 선형 확장 vs 실제 스피드업
ax2 = axes[1]
ideal = N_range
# 연산:통신 비율 = 4:1로 가정 (실제 GPU workload에 따라 다름)
comp_ratio = 4.0
speedup_ring = N_range / (1 + ring_comm / comp_ratio)
speedup_dp   = N_range / (1 + dp_master_comm / comp_ratio)

ax2.plot(N_range, ideal, 'g--', lw=1.5, label='이상적 선형 확장', alpha=0.6)
ax2.plot(N_range, speedup_ring, 'b-', lw=2.5, label='DDP (Ring All-Reduce)')
ax2.plot(N_range, speedup_dp, 'r-', lw=2.5, label='DP (마스터 중심)')
ax2.set_xlabel('GPU 수 (N)')
ax2.set_ylabel('학습 속도 향상 (배수)')
ax2.set_title('GPU 확장에 따른 실효 스피드업', fontweight='bold')
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("결론: DDP의 Ring All-Reduce는 GPU 수가 늘어도 통신 비용이 2K로 수렴하여")
print("      선형에 가까운 확장성(linear scaling)을 달성한다.")

## 4. Ring All-Reduce 심화

### 알고리즘 단계별 수식 도출

**설정**: GPU $N$개, 각각 크기 $K$의 텐서 $G_0, \dots, G_{N-1}$ 보유.  
**목표**: 모든 GPU가 $\bar{G} = \sum_{i} G_i$를 동일하게 보유

각 텐서를 $N$개의 청크로 분할: $G_i = [g_i^{(0)}, g_i^{(1)}, \dots, g_i^{(N-1)}]$

#### 1단계: Scatter-Reduce ($N-1$ 라운드)

- 라운드 $r$에서 GPU $i$는 청크 $(i - r) \bmod N$을 오른쪽 GPU로 전송
- 동시에 왼쪽에서 받은 청크를 자신의 해당 청크에 누적 합산
- 결과: 각 GPU는 정확히 하나의 청크에 대해 **전체 합산값** 보유

$$\text{GPU당 통신량}_{1단계} = (N-1) \times \frac{K}{N}$$

#### 2단계: All-Gather ($N-1$ 라운드)

- 완성된 합산 청크를 다시 링을 따라 전파
- 결과: 모든 GPU가 전체 합산 텐서를 완전히 복원

$$\text{GPU당 통신량}_{2단계} = (N-1) \times \frac{K}{N}$$

#### 최종 도출

$$T_{comm} = 2(N-1) \times \frac{K}{N} \xrightarrow{N \to \infty} 2K \quad \text{(상수 수렴)} $$

In [ ]:
# ---------------------------------------------------
# Ring All-Reduce 완전 구현 및 단계별 시각화
# ---------------------------------------------------

def ring_all_reduce(gradients, operation='sum', verbose=True):
    """
    Ring All-Reduce를 순수 파이썬 + NumPy로 완전 구현.
    
    Args:
        gradients (list of np.ndarray): N개 GPU의 그래디언트 텐서  
        operation (str): 'sum' 또는 'mean'
        verbose (bool): 단계별 로그 출력 여부
    Returns:
        results (list of np.ndarray): All-Reduce 완료된 텐서 (모든 GPU 동일)
    """
    N = len(gradients)
    K = gradients[0].size
    chunk_size = K // N
    assert K % N == 0, f"텐서 크기({K})가 GPU 수({N})로 나누어져야 합니다"

    # 딥 카피 (입력 변경 방지)
    buf = [[g[j*chunk_size:(j+1)*chunk_size].copy() for j in range(N)]
           for g in gradients]

    comm_bytes = 0

    if verbose:
        print(f"[Ring All-Reduce] GPU={N}, 텐서 크기={K}, 청크 크기={chunk_size}")
        print(f"이론 통신량: 2×({N}-1)×{K}/{N} = {2*(N-1)*K//N} 원소")

    # ── 1단계: Scatter-Reduce ──────────────────────────
    if verbose: print("\n[Phase 1] Scatter-Reduce:")
    for rnd in range(N - 1):
        for gpu in range(N):
            send_chunk  = (gpu - rnd) % N
            recv_chunk  = (gpu - rnd - 1) % N
            left_gpu    = (gpu - 1) % N
            # 왼쪽에서 받은 청크를 자신의 버퍼에 합산
            buf[gpu][recv_chunk] += buf[left_gpu][recv_chunk]
            comm_bytes += chunk_size
        if verbose:
            print(f"  Round {rnd+1}: 각 GPU가 청크 1개 송수신 및 합산 ✔")

    # ── 2단계: All-Gather ──────────────────────────────
    if verbose: print("\n[Phase 2] All-Gather:")
    for rnd in range(N - 1):
        for gpu in range(N):
            recv_chunk  = (gpu - rnd) % N
            left_gpu    = (gpu - 1) % N
            # 왼쪽의 완성된 청크를 복사
            buf[gpu][recv_chunk] = buf[left_gpu][recv_chunk].copy()
            comm_bytes += chunk_size
        if verbose:
            print(f"  Round {rnd+1}: 완성 청크 전파 ✔")

    results = [np.concatenate(buf[i]) for i in range(N)]
    if operation == 'mean':
        results = [r / N for r in results]

    if verbose:
        print(f"\n실제 통신량: {comm_bytes} 원소 (이론: {2*(N-1)*K//N} ✅)")
    return results


# ── 테스트 ──────────────────────────────────────────
N_gpus = 4
tensor_size = 8  # 8 / 4 = 2개 청크

grads = [np.random.randint(1, 10, tensor_size).astype(float) for _ in range(N_gpus)]
expected = sum(grads)

print("초기 그래디언트:")
for i, g in enumerate(grads):
    print(f"  GPU {i}: {g}")
print(f"기대 합산값: {expected}")

results = ring_all_reduce(grads)

print("\n=== 검증 ===")
all_ok = all(np.allclose(r, expected) for r in results)
for i, r in enumerate(results):
    print(f"  GPU {i}: {r}  {'✅' if np.allclose(r, expected) else '❌'}")
print(f"\n모든 GPU가 동일한 합산 결과: {'✅ 성공!' if all_ok else '❌ 실패!'}")

## 5. MirroredStrategy 실습

`tf.distribute.MirroredStrategy`는 단일 노드 내에서 DDP와 유사하게 동작합니다.

**핵심 규칙**:
1. 모델은 반드시 **`strategy.scope()` 내에서** 생성
2. 글로벌 배치 크기 = 로컬 배치 × 레플리카 수
3. 손실 합산은 **`tf.reduce_sum` (합산)** 사용 (평균 아님!)

In [ ]:
# ---------------------------------------------------
# MirroredStrategy MNIST 분산 학습
# ---------------------------------------------------

strategy = tf.distribute.MirroredStrategy()
NUM_REPLICAS = strategy.num_replicas_in_sync
print(f'레플리카(장치) 수: {NUM_REPLICAS}')

# 데이터 준비
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

BATCH_PER_REPLICA = 64
GLOBAL_BATCH = BATCH_PER_REPLICA * NUM_REPLICAS  # 분산 학습의 핵심 설정

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
            .shuffle(10000).batch(GLOBAL_BATCH).prefetch(2))
test_ds  = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
            .batch(GLOBAL_BATCH).prefetch(2))

print(f"글로벌 배치: {GLOBAL_BATCH} (레플리카당 {BATCH_PER_REPLICA})")
print(f"학습 스텝 수: {len(train_ds)}")

# 모델은 반드시 strategy.scope() 안에서 정의
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28, 28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)  # from_logits=True → 활성화 없음
    ], name='mnist_distributed')

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

model.summary()
print("\n→ strategy.scope() 안에서 생성된 모델은 각 장치에 레플리카로 자동 복제됩니다.")

In [ ]:
# 커스텀 학습 루프로 분산 내부 동작 관찰
# model.fit()은 내부에서 All-Reduce를 자동 수행하지만,
# 여기서는 그 흐름을 직접 제어하여 이해도를 높인다.

@tf.function
def train_step(inputs):
    x_batch, y_batch = inputs
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        # 분산 학습에서는 손실을 합산(sum)한 후 글로벌 배치 수로 나눔
        per_example_loss = tf.keras.losses.sparse_categorical_crossentropy(
            y_batch, logits, from_logits=True)
        loss = tf.reduce_sum(per_example_loss) / GLOBAL_BATCH  # ← 중요!
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

@tf.function
def distributed_train_step(inputs):
    # strategy.run이 각 레플리카에서 독립적으로 train_step 실행
    # 그래디언트는 내부에서 자동으로 All-Reduce 동기화됨
    per_replica_losses = strategy.run(train_step, args=(inputs,))
    # 각 레플리카 손실 합산
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)

dist_train_ds = strategy.experimental_distribute_dataset(train_ds)

# 1 에포크 수동 학습
total_loss = 0.0
num_batches = 0
for batch in dist_train_ds:
    loss = distributed_train_step(batch)
    total_loss += loss.numpy()
    num_batches += 1
    if num_batches % 200 == 0:
        print(f"  Step {num_batches}: avg_loss = {total_loss / num_batches:.4f}")

print(f"\n에포크 평균 손실: {total_loss / num_batches:.4f}")
print("→ strategy.run() 내부에서 각 레플리카가 독립 Forward/Backward 수행")
print("→ strategy.reduce(SUM)에서 NCCL All-Reduce로 그래디언트가 동기화됨")

In [ ]:
# model.fit()으로 나머지 학습 완성 (실제로는 이것이 더 간편)
history = model.fit(
    train_ds, epochs=4, validation_data=test_ds, verbose=1
)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, key, title in zip(axes, ['loss', 'accuracy'], ['손실(Loss)', '정확도(Accuracy)']):
    ax.plot(history.history[key], 'b-o', label='Train')
    ax.plot(history.history[f'val_{key}'], 'r-o', label='Val')
    ax.set_title(title)
    ax.set_xlabel('Epoch')
    ax.legend(); ax.grid(True, alpha=0.3)

plt.suptitle(f'MirroredStrategy ({NUM_REPLICAS}개 장치) MNIST', fontweight='bold')
plt.tight_layout()
plt.show()

## 6. 통신 비용 시각화

실제 대규모 클러스터에서 DP와 DDP의 네트워크 처리량 차이를 계산해 보자.

In [ ]:
# ---------------------------------------------------
# 실제 대역폭 기반 통신 시간 비교
# ---------------------------------------------------

# GPT-3 175B 그래디언트 크기 (FP32 기준)
model_params_B = 175  # 175B params
grad_size_GB = model_params_B * 4  # FP32: 4bytes × 175B ≈ 700 GB

# 네트워크 대역폭
nvlink_bandwidth_GBps = 600   # NVLink: 600 GB/s
infiniband_GBps       = 50    # InfiniBand HDR: 400 Gb/s ≈ 50 GB/s
ethernet_GBps         = 12.5  # 100GbE ≈ 12.5 GB/s

N_values = [2, 4, 8, 16, 32, 64, 128]

print(f"GPT-3 175B 그래디언트: {grad_size_GB} GB")
print(f"\n{'GPU수':>6} | {'Ring(NVLink)':>13} | {'Ring(IB)':>10} | {'DP 마스터(IB)':>14}")
print("-" * 50)

for N in N_values:
    ring_gb  = 2 * (N-1) * grad_size_GB / N
    ring_nv  = ring_gb / nvlink_bandwidth_GBps
    ring_ib  = ring_gb / infiniband_GBps
    dp_mast  = 2 * (N-1) * grad_size_GB / infiniband_GBps  # 마스터 병목
    print(f"{N:>6} | {ring_nv:>12.2f}s | {ring_ib:>9.1f}s | {dp_mast:>13.1f}s")

print("\n→ 128-GPU 기준, Ring All-Reduce(IB)는 ≈14초, DP 마스터는 ≈1792초 (128배 차이!)")

## 7. 정리

### 핵심 개념 요약

| 항목 | Data Parallelism (DP) | Distributed Data Parallel (DDP) |
|------|----------------------|---------------------------------|
| 아키텍처 | 단일 프로세스 + 멀티 스레드 | 멀티 프로세스 (GPU당 1개) |
| 통신 구조 | 마스터 중심 Fan-in/out | 링(Ring) P2P |
| 마스터 병목 | $O(N \times K)$ → ✗ | 없음 → ✅ |
| GPU당 통신량 | $2(N-1)K$ → 선형 증가 | $2(N-1)K/N \approx 2K$ → 상수 |
| GIL 영향 | 있음 | 없음 |
| 멀티 노드 | 불가 | 가능 |

### 핵심 수식

$$T_{Ring} = 2(N-1)\frac{K}{N} \xrightarrow{N \to \infty} 2K, \qquad M_{training}^{Adam} = 16P \text{ bytes}$$

### 다음 챕터 예고
**Chapter 09-02: 텐서 병렬화 (Tensor Parallelism)** — 모델이 단일 GPU 메모리에 통째로 올라가지 않을 때, 가중치 행렬 자체를 열(Column)/행(Row) 방향으로 분할하여 여러 GPU에 나누는 Megatron-LM 방식의 완전한 수학적 증명과 NumPy 시뮬레이션을 학습한다.